In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import warnings
import numpy as np

In [16]:
# Carregar os dados do arquivo Excel
dados = pd.read_excel("variaveis.xlsx")

# Limpar espaços em branco em colunas categóricas
categoricas = ['Qntd_Finais_de_Semana', 'Feriados_Prolongados', 'Feriados_Meio_Semana', 'Ferias_Escolares', 'Datas_Sazonais', 'Datas_de_Liquidacao_Interna', 'Ressacas']

for column in categoricas:
    if dados[column].dtype == 'object':
        dados[column] = dados[column].str.strip()  # Remover espaços em branco no início e no final

# Separar as variáveis independentes (X) e a variável dependente (y)
y = dados['Vendas']
X = dados.drop(['Data', 'Vendas'], axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Inicializar um dicionário para armazenar os resultados dos modelos
resultados = {}

# Ignorar os avisos sobre os nomes das características
warnings.filterwarnings("ignore", category=UserWarning)

# Regressão Linear
regressao = LinearRegression()
regressao.fit(X_train, y_train)
y_pred_lr = regressao.predict(X_test)
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
resultados['Regressão Linear'] = rmse_lr

# Gradient Boosting
gradient_boosting = GradientBoostingRegressor(random_state=0)
gradient_boosting.fit(X_train, y_train)
y_pred_gb = gradient_boosting.predict(X_test)
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)
resultados['Gradient Boosting'] = rmse_gb

# Rede Neural
redes_neurais = MLPRegressor(random_state=0, max_iter=1000)
redes_neurais.fit(X_train, y_train)
y_pred_mlp = redes_neurais.predict(X_test)
rmse_mlp = mean_squared_error(y_test, y_pred_mlp, squared=False)
resultados['Neural Network'] = rmse_mlp

# Random Forest
random_forest = RandomForestRegressor(random_state=0)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
resultados['Random Forest'] = rmse_rf

# Imprimir os resultados dos modelos
resultados


{'Regressão Linear': 5615804.235092652,
 'Gradient Boosting': 6108056.886356479,
 'Neural Network': 35673675.84347872,
 'Random Forest': 6949318.312419684}

Esse código realiza os seguintes passos:

1) Leitura dos Dados: Carrega os dados de um arquivo Excel usando o Pandas. 
2) Limpeza de Dados: Remove espaços em branco no início e no final das colunas categóricas para evitar erros. 
3) Preparação dos Dados: Separa as variáveis independentes (X) e a variável dependente (y). 
Divisão dos Dados: Divide os dados em conjuntos de treinamento e teste.
4) Modelagem e Avaliação: Treina modelos de Regressão Linear, Gradient Boosting, Rede Neural e Random Forest, e avalia o desempenho de cada modelo usando a raiz quadrada do erro médio quadrático (RMSE). 
5) Resultados: Armazena os resultados em um dicionário chamado resultados e imprime os resultados finais. 

Obs.
Os resultados fornecem a avaliação do desempenho dos modelos treinados com base na métrica de raiz quadrada do erro médio quadrático (RMSE) nos dados de teste, indicando a precisão do modelo. Um RMSE mais baixo indica melhor desempenho. Nesse caso, os valores dos resultado indicam um baixo desempenho, para obter melhores resultados, recomenda-se a inserção de mais dados para o modelo entender melhor a estrutura subjacente dos dados e consequentemente melhorar sua capacidade de previsão.

In [17]:
# Definir os valores para outubro de 2023
valores_outubro_2023 = np.array([[5, 1, 0, 0, 1, 0, 0]])

# Fazer previsões utilizando os modelos treinados
pred_lr_outubro = regressao.predict(valores_outubro_2023)
pred_gb_outubro = gradient_boosting.predict(valores_outubro_2023)
pred_mlp_outubro = redes_neurais.predict(valores_outubro_2023)
pred_rf_outubro = random_forest.predict(valores_outubro_2023)

# Armazenar as previsões em um dicionário
previsoes_outubro = {
    'Linear Regression': pred_lr_outubro[0],
    'Gradient Boosting': pred_gb_outubro[0],
    'Neural Network': pred_mlp_outubro[0],
    'Random Forest': pred_rf_outubro[0]
}

# Imprimir as previsões
print("Previsões para Outubro de 2023:")
for modelo, previsao in previsoes_outubro.items():
    print(f'{modelo}: {previsao:.2f}')

# Comparar com o valor real de vendas
valor_real_outubro = 32678694.07

# Imprimir a comparação com o valor real
print("\nComparação com o Valor Real:")
for modelo, previsao in previsoes_outubro.items():
    erro_absoluto = abs(previsao - valor_real_outubro)
    print(f'{modelo}: Erro Absoluto = {erro_absoluto:.2f}')


Previsões para Outubro de 2023:
Linear Regression: 36406070.34
Gradient Boosting: 33255332.03
Neural Network: 1002.67
Random Forest: 32666386.40

Comparação com o Valor Real:
Linear Regression: Erro Absoluto = 3727376.27
Gradient Boosting: Erro Absoluto = 576637.96
Neural Network: Erro Absoluto = 32677691.40
Random Forest: Erro Absoluto = 12307.67


O modelo Random Forest foi o mais eficiente em prever os valores do faturamento total do Outlet, com um erro absoluto de aproximadamente 12,3 mil reais, apenas 0.04% em relação ao valor real de aproximidamente 32,6 milhões.

In [18]:
def fazer_previsoes(valores, modelos):
    # Fazer previsões utilizando os modelos treinados
    previsoes = {}
    for modelo, regressor in modelos.items():
        predicao = regressor.predict(valores)
        previsoes[modelo] = predicao[0]

    # Imprimir as previsões
    print("Previsões:")
    for modelo, previsao in previsoes.items():
        print(f'{modelo}: {previsao:.2f}')

    # Se desejar, você pode retornar as previsões para uso posterior
    return previsoes

# Exemplo de definição de modelos
modelos = {
    'Linear Regression': regressao,
    'Gradient Boosting': gradient_boosting,
    'Neural Network': redes_neurais,
    'Random Forest': random_forest
}

# Exemplo de valores ['Qntd_Finais_de_Semana', 'Feriados_Prolongados', 'Feriados_Meio_Semana', 'Ferias_Escolares', 'Datas_Sazonais', 'Datas_de_Liquidacao_Interna', 'Ressacas']
# 'Qntd_Finais_de_Semana': 4 ou 5
# 'Feriados_Prolongados', 'Feriados_Meio_Semana', 'Ferias_Escolares', 'Datas_Sazonais', 'Datas_de_Liquidacao_Interna', 'Ressacas': 1 para Sim e 0 para Não

valores_para_prever = np.array([[5, 1, 0, 0, 1, 0, 0]]) # Inserir aqui os valores

# Fazer previsões para os valores fornecidos
previsoes = fazer_previsoes(valores_para_prever, modelos)


Previsões:
Linear Regression: 36406070.34
Gradient Boosting: 33255332.03
Neural Network: 1002.67
Random Forest: 32666386.40
